<a href="https://colab.research.google.com/github/icehelmetminer/DISASTERS/blob/main/supplychain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [174]:
import pandas as pd
data_xls = pd.read_excel('/content/savefile_workingproject.xlsx', 'Sheet1', dtype=str, index_col=None)
data_xls.to_csv('csvfile.csv', encoding='utf-8', index=False)
dataset = pd.read_csv('csvfile.csv')

In [175]:
dataset.head()


,serial_numbers,serial,customer_id,customer,dest,shipped,returned,customer_possession_time
0,SN-0000767,9,553240,553240,TAIWAN,2013-09-18 00:00:00,2018-02-07 00:00:00,1602.636806
1,SN-0001178,41,545473,545473,SE ASIA,2013-09-20 00:00:00,2016-06-21 00:00:00,1005.411806
2,SN-0000616,150,550505,550505,TAIWAN,2013-09-30 00:00:00,2015-10-15 00:00:00,744.669444
3,SN-0000339,1919,553166,553166,TAIWAN,2013-10-18 00:00:00,2014-04-29 00:00:00,193.000000
4,SN-0000325,1210,553166,553166,TAIWAN,2013-10-18 00:00:00,2014-03-18 00:00:00,151.000000


In [176]:
dataset.shipped=pd.to_datetime(dataset.shipped)
dataset.returned=pd.to_datetime(dataset.returned)
dataset['duration']=dataset.returned-dataset.shipped
dataset

,serial_numbers,serial,customer_id,customer,dest,shipped,returned,customer_possession_time,duration
0,SN-0000767,9,553240,553240,TAIWAN,2013-09-18,2018-02-07,1602.636806,1603 days
1,SN-0001178,41,545473,545473,SE ASIA,2013-09-20,2016-06-21,1005.411806,1005 days
2,SN-0000616,150,550505,550505,TAIWAN,2013-09-30,2015-10-15,744.669444,745 days
3,SN-0000339,1919,553166,553166,TAIWAN,2013-10-18,2014-04-29,193.000000,193 days
4,SN-0000325,1210,553166,553166,TAIWAN,2013-10-18,2014-03-18,151.000000,151 days
...,...,...,...,...,...,...,...,...,...
6376,002782N,2506,534221,534221,CHINA,2020-02-24,2020-03-06,11.066667,11 days
6377,000242N,1329,550505,550505,TAIWAN,2020-02-25,2020-04-30,64.203472,65 days
6378,001511N,1943,550505,550505,TAIWAN,2020-03-08,2020-04-30,52.209722,53 days
6379,SN-0004225,1074,553166,553166,TAIWAN,2020-03-18,2020-05-26,68.114583,69 days


In [177]:
p_start=dataset.shipped.min()
p_end=dataset.returned.max()
p_duration=(p_end-p_start).days+1
print('project duration :', p_duration)
print('project start date :', p_start)
print('project end date :', p_end)

project duration : 2445
project start date : 2013-09-18 00:00:00
project end date : 2020-05-28 00:00:00


In [178]:
dataset.sort_values(by='shipped', ascending=True, na_position='first')

,serial_numbers,serial,customer_id,customer,dest,shipped,returned,customer_possession_time,duration
0,SN-0000767,9,553240,553240,TAIWAN,2013-09-18,2018-02-07,1602.636806,1603 days
1,SN-0001178,41,545473,545473,SE ASIA,2013-09-20,2016-06-21,1005.411806,1005 days
2,SN-0000616,150,550505,550505,TAIWAN,2013-09-30,2015-10-15,744.669444,745 days
3,SN-0000339,1919,553166,553166,TAIWAN,2013-10-18,2014-04-29,193.000000,193 days
4,SN-0000325,1210,553166,553166,TAIWAN,2013-10-18,2014-03-18,151.000000,151 days
...,...,...,...,...,...,...,...,...,...
6376,002782N,2506,534221,534221,CHINA,2020-02-24,2020-03-06,11.066667,11 days
6377,000242N,1329,550505,550505,TAIWAN,2020-02-25,2020-04-30,64.203472,65 days
6378,001511N,1943,550505,550505,TAIWAN,2020-03-08,2020-04-30,52.209722,53 days
6379,SN-0004225,1074,553166,553166,TAIWAN,2020-03-18,2020-05-26,68.114583,69 days


In [179]:
#dataset['container_status'] = ''


dataset.head(2)

,serial_numbers,serial,customer_id,customer,dest,shipped,returned,customer_possession_time,duration
0,SN-0000767,9,553240,553240,TAIWAN,2013-09-18,2018-02-07,1602.636806,1603 days
1,SN-0001178,41,545473,545473,SE ASIA,2013-09-20,2016-06-21,1005.411806,1005 days


In [180]:
#!pip install icecream
def initializer(length): # this adds empty array to df
    array = []
    value = ''
    for i in range(length):
        array.append(value)
    return array

In [181]:
#!pip install icecream
def initializer2(length): # this adds empty array to df
    array = []
    for i in range(length):
        array.append(i)
    return array

In [188]:
import icecream 
from icecream import ic
import time
import numpy as np
#!pip install tqdm
import tqdm
from tqdm import tqdm
#has customer k received unit i in the past? 
the_container_store = pd.DataFrame()
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
dataset['index'] = initializer2(len(dataset['serial'])) 

the_container_store['unit'] = initializer(len(dataset['serial'])) # SN-0000767
the_container_store['number_of_customer_buys'] = initializer(len(dataset['serial'])) # [553240,545473]
the_container_store['customer_buys'] = initializer(len(dataset['serial'])) # [553240,545473]
the_container_store['customer_id'] = initializer(len(dataset['serial'])) # [553240,545473]
the_container_store['customer_bitems'] = initializer(len(dataset['serial'])) # [553240,545473]


the_container_store['customer_possession_time'] = initializer(len(dataset['serial'])) #[average of 1602.636806 and 1005.411806]

counter = 0

items = dataset['serial']
'''
 	serial_numbers 	serial 	customer_id 	customer 	dest 	shipped 	returned 	customer_possession_time 	duration
0 	SN-0000767 	9 	553240 	553240 	TAIWAN 	2013-09-18 	2018-02-07 	1602.636806 	1603 days

'''
#2509 units 
#counter is the index 
import statistics
import math

counter = 0

items = dataset['serial']

dataset = dataset[['index','serial','customer_id','customer','customer_possession_time']]


for item in tqdm(items):
  #ic(item)
  the_container_store['unit'][counter] = item
  #ic(the_container_store)
  buyer = dataset['customer'][counter]
  return_time = dataset['customer_possession_time'][counter]
  buys = dataset.loc[dataset['customer'] == buyer] # all the places the buyer is listed in the customer column 
  bitems = dataset.loc[dataset['serial'] == item]
  buys_array = np.array(buys.index.values)
  bitems_array = np.array(bitems.index.values)

  common_indices = list(np.intersect1d(buys_array, bitems_array))

  #print(f'\nchecking for accuracy \n{buyer} is the buyer id for transactions involving the following items:\n')
  common_df = []
  returnrow = []
  return_time = 0
  for ind in common_indices:
    # show the transactions
    dataset.rename(columns=dataset.iloc[0]).drop(dataset.index[0])
    row = dataset.loc[dataset['index'] == ind]
    row = row.values.tolist()
    row_inner = row[0]
    row_superinner = row_inner[0]
    returnrow.append(row_superinner)
    return_time = len(row_inner)/sum(row_inner)
    #print(len(row_inner))
    #print(sum(row_inner))
  the_container_store['customer_id'][counter] = buyer
  the_container_store['customer_buys'][counter] = returnrow
  the_container_store['number_of_customer_buys'][counter] = len(returnrow)
  the_container_store['customer_possession_time'][counter] = statistics.mean(returnrow)
  counter+=1

the_container_store.head()
the_container_store.to_csv("quicksave.csv")
#the_container_store['customer']
ic()

100%|██████████| 6381/6381 [00:33<00:00, 190.25it/s]
ic| <ipython-input-188-05ce07ead4f4>:81 in <module> at 06:45:08.424


In [182]:
#the_container_store['unit'] = dataset['customer_id']
'''old_customer_entry = np.array(the_container_store['customer'][counter])

current_customer_array = np.array(dataset['customer_id'][counter])
#new_customer_entry = np.vstack((old_customer_entry,current_customer_array))
new_customer_entry = old_customer_entry.append(old_customer_entry)
new_customer_entry = np.concatenate(old_customer_entry,current_customer_array)
the_container_store['customer'][counter] = new_customer_entry
the_container_store['possession_times'][counter] = dataset['customer_id'][counter]

'''

#print(the_container_store)
'''testing=[]
for i in range(3):
   testing.append([1,2,3])
'''



'''for container in tqdm(items): # for every unit we want to look at which customers bought it 
  #container is a # index 
  for customer in dataset['customer']:
    #print(customer," bought and returned in ",return_time," days")
    try:
      return_time = dataset['customer_possession_time'][counter]
    except Exception:
      continue'''
    
    #counter+=1

'''

for container in tqdm(items):
  for customer,return_time in zip(dataset['customer'],dataset['customer_possession_time']):
    the_container_store['unit'][counter] = container
    customers = the_container_store['customer'][counter] 
    try:
      customers = pd.array(customers)
      customers.append(customer)
    except Exception as e:
      #print(e)
      continue
    the_container_store['possession_times'][counter] = return_time
    counter+=1'''

  #for customer in dataset['']

"\n\nfor container in tqdm(items):\n  for customer,return_time in zip(dataset['customer'],dataset['customer_possession_time']):\n    the_container_store['unit'][counter] = container\n    customers = the_container_store['customer'][counter] \n    try:\n      customers = pd.array(customers)\n      customers.append(customer)\n    except Exception as e:\n      #print(e)\n      continue\n    the_container_store['possession_times'][counter] = return_time\n    counter+=1"

In [183]:
the_container_store.head(20)
items

0          9
1         41
2        150
3       1919
4       1210
        ... 
6376    2506
6377    1329
6378    1943
6379    1074
6380      89
Name: serial, Length: 6381, dtype: int64